============================================================================================

# Kmeans

============================================================================================


In [1]:
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
import pandas as pd
import numpy as np
import os
import re
import random
import time
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA

import fns_models as fns


% matplotlib inline

from subprocess import check_output
print(check_output(["ls", "data"]).decode("utf-8"))

athenaeum_authors_preview.csv
athenaeum_painting_filtered.csv
athenaeum_paintings.csv
athenaeum_paintings_sizes.csv
color_histograms.csv
complete_data.csv
extra_tree_com.csv
grad_boost_com.csv
images
images_athenaeum
images_sizes_2325.csv
nbc_com.csv
net_predicted.csv
painter_info_clean.csv
painting_info_clean.csv
resized_200
rf_com.csv
test_author200.csv
test_data.csv
test_hist_author_knn.csv
test_hist_author_rf.csv
train_author200.csv
train_data.csv
train_hist_author_knn.csv
train_hist_author_rf.csv
xgb_com.csv



In [8]:
color_hist = pd.read_csv('data/color_hist_size_206552.csv')

In [9]:
color_hist.head(3)

,author_id,painting_id,hist_01,hist_02,hist_03,hist_04,hist_05,hist_06,hist_07,hist_08,...,hist_24,hist_25,hist_26,hist_27,hist_28,hist_29,hist_30,height_px,width_px,height_width_ratio
0,444,12077,7603,1687,587,348,403,538,671,959,...,69539,24143,46874,97427,117568,102332,79799,555.0,800.0,0.693750
1,444,11653,141479,107241,3576,1031,1010,3886,568,2650,...,99116,46843,25371,63071,67621,69665,480656,742.0,952.0,0.779412
2,444,12097,1429,713,600,581,1362,3304,8280,16148,...,107953,51343,77403,128000,147619,172075,150541,941.0,718.0,1.310585


In [10]:
color_hist.drop(['height_px', 'width_px', 'author_id'], axis=1)

,painting_id,hist_01,hist_02,hist_03,hist_04,hist_05,hist_06,hist_07,hist_08,hist_09,...,hist_22,hist_23,hist_24,hist_25,hist_26,hist_27,hist_28,hist_29,hist_30,height_width_ratio
0,12077,7603,1687,587,348,403,538,671,959,1919,...,83566,99139,69539,24143,46874,97427,117568,102332,79799,0.693750
1,11653,141479,107241,3576,1031,1010,3886,568,2650,5094,...,117472,144006,99116,46843,25371,63071,67621,69665,480656,0.779412
2,12097,1429,713,600,581,1362,3304,8280,16148,39395,...,132292,174675,107953,51343,77403,128000,147619,172075,150541,1.310585
3,150612,20842,9295,1219,1709,1595,1675,2465,3657,5312,...,150594,64212,9829,1196,14227,60984,83664,86097,32933,0.846422
4,12090,300,309,636,2093,6897,15093,15927,20497,30978,...,146185,88366,60395,37406,8898,38751,90275,176120,87056,0.818571
5,23129,224669,135759,14790,4682,3749,46189,2445,3384,4113,...,153033,65668,9887,757,37788,150881,122183,98409,163171,0.494424
6,11492,3657,3572,4864,5715,5598,5146,4496,9854,30311,...,208795,199168,50242,9207,111087,220867,185409,35095,342,1.157742
7,12088,4874,4434,1560,1094,1058,1046,930,1279,1875,...,163396,59144,21843,8734,10406,27220,80750,161781,57565,1.509514
8,65526,6183,650,172,50,115,244,96,476,6851,...,59693,141606,298087,365634,69259,245911,370106,188673,326645,0.829593
9,12074,6993,11865,6386,1813,5310,3991,3114,8577,24802,...,22114,45949,87153,57901,44446,41484,36042,120426,139682,0.414583


In [13]:
kmeans = KMeans()
kmeans.set_params(n_clusters=7)
kmeans.fit(color_hist.iloc[:, 1:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [15]:
kmeans.labels_

array([6, 0, 6, ..., 3, 4, 0], dtype=int32)

In [16]:
(kmeans.labels_).shape

(206552,)

In [17]:
color_hist['kmeans_labels'] = kmeans.labels_

In [18]:
color_hist.head(1)

,author_id,painting_id,hist_01,hist_02,hist_03,hist_04,hist_05,hist_06,hist_07,hist_08,...,hist_25,hist_26,hist_27,hist_28,hist_29,hist_30,height_px,width_px,height_width_ratio,kmeans_labels
0,444,12077,7603,1687,587,348,403,538,671,959,...,24143,46874,97427,117568,102332,79799,555.0,800.0,0.69375,6


In [19]:
color_hist.shape

(206552, 36)

In [20]:
color_hist.to_csv('data/color_hist_kmeans_206552.csv', index=False)

In [21]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
tsne.fit_transform(color_hist.iloc[:, 1:]) 